In [1]:
import os
from pathlib import Path

In [2]:
TIMEFRAME = "15min"
PAIR = "BTCUSDT"
DATA_PATH = Path.home() / "data" / PAIR
TMP_PATH = Path("./tmp/").resolve()
CONFIG_PATH = Path("./config/").resolve()
LOG_PATH = Path("./log/").resolve()
DATA_PATH.mkdir(exist_ok=True, parents=True)
TMP_PATH.mkdir(exist_ok=True, parents=True)
CONFIG_PATH.mkdir(exist_ok=True, parents=True)
LOG_PATH.mkdir(exist_ok=True, parents=True)

Save: OHLCV + Open Interest from Bybit API

In [3]:
import ccxt
from dotenv import load_dotenv
load_dotenv(verbose=True)
dotenv_path = Path.home() / ".env"
load_dotenv(dotenv_path)
exchange = ccxt.bybit()
exchange.apiKey = os.environ["BYBIT_API_KEY"]
exchange.secret = os.environ["BYBIT_SECRET"]
exchange.options["timeDifference"] = 5000

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
ohlcv = exchange.fetch_ohlcv(PAIR, "15m", limit=200)
oi = exchange.fetch_open_interest_history(PAIR, "15min", limit=200)
oi = [[int(d["info"]["timestamp"]) * 1000, float(d["info"]["open_interest"])] for d in oi]
assert (len(ohlcv) == len(oi))
assert (ohlcv[0][0] == oi[0][0] and ohlcv[-1][0] == oi[-1][0])
data = [a + b[1:] for a, b in zip(ohlcv, oi)]
df = pd.DataFrame(data, columns=["Datetime", "Open", "High", "Low", "Close", "Volume", "OpenInterest"])
df["Datetime"] = pd.to_datetime(df["Datetime"], unit="ms")
df = df.set_index("Datetime")
df

,Open,High,Low,Close,Volume,OpenInterest
Datetime,,,,,,
2022-08-17 03:30:00,24031.0,24053.5,24011.5,24020.0,553.493,45105.596
2022-08-17 03:45:00,24020.0,24025.0,23984.5,23989.5,784.358,45109.668
2022-08-17 04:00:00,23989.5,24018.5,23989.5,23997.0,272.634,45212.083
2022-08-17 04:15:00,23997.0,24008.0,23996.0,24000.0,281.682,45150.676
2022-08-17 04:30:00,24000.0,24039.0,23994.5,24011.5,342.948,45163.414
...,...,...,...,...,...,...
2022-08-19 04:15:00,22771.0,22805.5,22761.0,22793.5,634.003,52253.890
2022-08-19 04:30:00,22793.5,22823.0,22793.0,22793.5,833.398,52259.122
2022-08-19 04:45:00,22793.5,22806.0,22779.0,22779.5,470.622,52594.824


In [17]:
from scripts.extract_features import attach_features
df = attach_features(df)
oi = df["OpenInterest"].apply(np.log1p)
for ts in [1, 5, 10, 20]:
    df[f"feature_oi_log_return_{ts}"] = oi.diff(ts)
df

,Open,High,Low,Close,Volume,OpenInterest,feature_candle_value,feature_candle_value_mean_10,feature_candle_value_mean_20,feature_candle_value_mean_5,...,feature_upper_shadow_mean_5,feature_volatility_10,feature_volatility_20,feature_volatility_3,feature_volatility_5,feature_volatility_50,feature_oi_log_return_1,feature_oi_log_return_5,feature_oi_log_return_10,feature_oi_log_return_20
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-08-17 16:00:00,23383.5,23407.0,23326.0,23358.0,1083.235,48760.001,-0.314815,-0.055994,-0.178659,0.117583,...,0.001312,0.001807,0.002186,0.001486,0.002106,0.003016,NaN,NaN,NaN,NaN
2022-08-17 16:15:00,23358.0,23393.0,23331.5,23371.5,1012.753,49336.439,0.219512,0.004696,-0.140350,-0.003631,...,0.001402,0.001648,0.002202,0.000940,0.001470,0.003018,0.011752,NaN,NaN,NaN
2022-08-17 16:30:00,23371.5,23450.0,23355.0,23410.0,2034.795,49435.294,0.405263,0.073318,-0.100522,-0.004546,...,0.001582,0.001691,0.002268,0.001379,0.001618,0.003031,0.002002,NaN,NaN,NaN
2022-08-17 16:45:00,23410.0,23432.0,23384.0,23408.0,811.339,49388.881,-0.041667,0.050164,-0.098666,0.103121,...,0.001206,0.001681,0.002270,0.000874,0.001001,0.003031,-0.000939,NaN,NaN,NaN
2022-08-17 17:00:00,23408.0,23428.0,23359.5,23387.0,1027.753,49789.910,-0.306569,0.066445,-0.096494,-0.007655,...,0.001501,0.001570,0.002266,0.001299,0.001123,0.003028,0.008087,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-19 04:15:00,22771.0,22805.5,22761.0,22793.5,634.003,52253.890,0.505618,0.069551,-0.117658,0.131197,...,0.000659,0.001587,0.002230,0.001038,0.001025,0.002058,0.002601,0.010398,0.030838,0.030821
2022-08-19 04:30:00,22793.5,22823.0,22793.0,22793.5,833.398,52259.122,0.000000,0.060909,-0.128634,-0.028803,...,0.000838,0.001587,0.002207,0.000497,0.000738,0.002054,0.000100,0.007453,0.027450,0.030760
2022-08-19 04:45:00,22793.5,22806.0,22779.0,22779.5,470.622,52594.824,-0.518519,-0.048490,-0.139139,-0.125489,...,0.000895,0.001542,0.002208,0.000808,0.000800,0.002012,0.006403,0.008573,0.029919,0.048568


In [3]:
from ray.rllib.examples.env.simple_rpg import SimpleRPG
from ray.rllib.examples.models.simple_rpg_model import CustomTorchRPGModel

In [52]:
import numpy as np
import gym
from gym import spaces
from ray.rllib.utils.spaces.repeated import Repeated
class SimpleTradingEnv(gym.Env):
    def __init__(self, config = None):
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Dict(
            {
                "position": spaces.Box(-np.inf, np.inf, shape=(2,), dtype=np.float32),
                "short": spaces.Box(
                    -np.inf,
                    np.inf,
                    shape=(5, 3),
                    dtype=np.float32
                ),
                "middle": spaces.Box(
                    -np.inf,
                    np.inf,
                    shape=(5, 3),
                    dtype=np.float32,
                ),
                "long": spaces.Box(
                    -np.inf,
                    np.inf,
                    shape=(5, 3),
                    dtype=np.float32,
                ),
                # "middle": self.features_space,
                # "long": self.features_space,
            }
        )

    def reset(self):
        return self.observation_space.sample()

    def step(self, action):
        return self.observation_space.sample(), 0, True, {}

In [53]:
import torch
import torch.nn as nn
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork 
class MultiTimeframeModel(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        super().__init__(obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        self.model = FullyConnectedNetwork(obs_space, action_space, num_outputs, model_config, name)

    def forward(self, input_dict, state, seq_lens):
        print("The unpacked input tensors:", input_dict["obs"])
        print()
        # print("Unbatched repeat dim", input_dict["obs"].unbatch_repeat_dim())
        # print()
        # print("Fully unbatched", input_dict["obs"].unbatch_all())
        # print()
        return self.model.forward(input_dict, state, seq_lens)

    def value_function(self):
        return self.model.value_function()

In [54]:
from ray.rllib.models import ModelCatalog
ModelCatalog.register_custom_model("multi_timeframe_model", MultiTimeframeModel)
config = {
    "framework": "torch",
    "env": SimpleTradingEnv,
    "model": {
        "custom_model": "multi_timeframe_model",
    },
    "rollout_fragment_length": 1,
    "train_batch_size": 2,
    "num_workers": 0,
    "_disable_preprocessor_api": False,
}

In [55]:
from ray.rllib.agents import dqn, pg
# agent = dqn.DQNTrainer(config)
agent = pg.PGTrainer(config)

2022-08-19 17:53:33,493	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


The unpacked input tensors: OrderedDict([('long', tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 

In [57]:
# env = SimpleRPG({})
from pprint import pprint
env = SimpleTradingEnv({})
obs = env.reset()
done = False
while not done:
    # action = env.action_space.sample()
    action = agent.compute_action(obs)
    obs, reward, done, info = env.step(action)
    pprint(obs)
    # for k, v in obs.items():
    #     print(k, v)

The unpacked input tensors: OrderedDict([('long', tensor([[[-0.2535,  1.0208, -0.2529],
         [-0.8066, -1.3033,  0.3201],
         [ 1.3316,  0.9512, -0.1843],
         [-0.7106, -1.1891, -1.0919],
         [-1.2125,  1.7145,  1.7286]]])), ('middle', tensor([[[-8.5962e-01,  5.4824e-01,  7.7150e-01],
         [ 2.6335e-01, -2.3191e-01, -2.6616e-01],
         [-1.2190e+00, -1.1456e+00,  4.0465e-01],
         [-3.5876e-01,  4.6316e-04,  1.4859e+00],
         [ 6.1718e-01, -6.5388e-01,  5.0607e-01]]])), ('position', tensor([[-0.6601,  0.3746]])), ('short', tensor([[[ 1.7032,  1.6657, -0.4031],
         [-0.1881,  0.7004,  2.2929],
         [ 0.2479,  2.1643, -2.1120],
         [-2.9329,  0.0396,  0.1426],
         [ 0.3430,  0.1579, -0.6221]]]))])

OrderedDict([('long',
              array([[-0.1931002 ,  0.52399224,  1.2975646 ],
       [-2.1278567 , -0.25202775,  0.37695202],
       [-1.0774863 ,  0.8742588 , -0.54340786],
       [ 0.19233477,  0.6199103 ,  0.28522074],
       [ 1.40